In [1]:
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from keras import backend as K

from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Sequential
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from keras_helper import NNWeightHelper
from snes import SNES
from sklearn.ensemble import RandomForestClassifier
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import os
import cv2
import matplotlib.pyplot as plt
from dataSet import dataSet

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28
num_classes = 31

In [4]:
# use just a small sample of the train set to test
SAMPLE_SIZE = 1024
# how many different sets of weights ask() should return for evaluation
POPULATION_SIZE = 93
# how many times we will loop over ask()/tell()
GENERATIONS = 100

In [5]:
Amazon_path = './Original_images/amazon/images'
dslr_path   = './Original_images/dslr/images'
webcam_path = './Original_images/webcam/images'
paths = [Amazon_path, dslr_path, webcam_path]
files = os.listdir(Amazon_path)
labels = {}
count  = 0
for key in files:
    a = {key : count}
    labels.update(a)
    count += 1
print (labels)

images_path = []
Amazon = dataSet()
dslr   = dataSet()
webcam = dataSet()

{'mobile_phone': 0, 'paper_notebook': 1, 'back_pack': 2, 'monitor': 3, 'projector': 4, 'calculator': 5, 'desk_lamp': 6, 'laptop_computer': 7, 'speaker': 8, 'letter_tray': 9, 'bike': 10, 'trash_can': 11, 'printer': 12, 'stapler': 13, 'headphones': 14, 'punchers': 15, 'file_cabinet': 16, 'tape_dispenser': 17, 'mouse': 18, 'pen': 19, 'bookcase': 20, 'desk_chair': 21, 'desktop_computer': 22, 'ruler': 23, 'mug': 24, 'phone': 25, 'scissors': 26, 'ring_binder': 27, 'bike_helmet': 28, 'bottle': 29, 'keyboard': 30}


In [6]:
for dirname in files:
    images_name = os.listdir(Amazon_path + '/' + dirname)
    for name in images_name:
        Image_Path = Amazon_path + '/' + dirname + '/' + name
        images_path.append(Image_Path)
        image_data = cv2.imread(Image_Path)
        image_data = cv2.resize(image_data, (img_rows, img_cols), interpolation=cv2.INTER_CUBIC)
        image_data = image_data.reshape(img_rows, img_cols, 3)
        Amazon.upData(image_data, labels[dirname], labels)        

Amazon.sHape()


done


In [7]:
Amazon.data.shape

(2817, 2352)

In [8]:
for dirname in files:
    images_name = os.listdir(dslr_path + '/' + dirname)
    for name in images_name:
        Image_Path = dslr_path + '/' + dirname + '/' + name
        images_path.append(Image_Path)
        image_data = cv2.imread(Image_Path)
        image_data = cv2.resize(image_data, (img_rows, img_cols), interpolation=cv2.INTER_CUBIC)
        image_data = image_data.reshape(img_rows, img_cols, 3)
        dslr.upData(image_data, labels[dirname], labels)
dslr.sHape()


done


In [9]:
for dirname in files:
    images_name = os.listdir(webcam_path + '/' + dirname)
    for name in images_name:
        Image_Path = webcam_path + '/' + dirname + '/' + name
        images_path.append(Image_Path)
        image_data = cv2.imread(Image_Path)
        image_data = cv2.resize(image_data, (img_rows, img_cols), interpolation=cv2.INTER_CUBIC)
        image_data = image_data.reshape(img_rows, img_cols, 3)
        webcam.upData(image_data, labels[dirname], labels)
webcam.sHape()


done


In [12]:
def train_classifier(model, X, y):
    X_features = model.predict(X)
    clf = RandomForestClassifier()
    clf.fit(X_features, y)
    y_pred = clf.predict(X_features)
    return clf, y_pred
def predict_classifier(model, clf, X):
    X_features = model.predict(X)
    return clf.predict(X_features)
# input image dimensions
img_rows, img_cols = 28, 28
num_classes = 10

# Load Amazon for target data
x_train = Amazon.data
y_train = Amazon.label

# load DSLR for source data
x_test = dslr.data
y_test = dslr.label

#domain data
x_train_domain = np.concatenate((x_train,x_test), axis=0)
y_train_domain = np.concatenate((np.zeros(x_train.shape[0]), np.ones(x_test.shape[0])),axis=0)

# x_test_domain = np.concatenate((x_test1,mnistm_test), axis=0)# totaly 20000 sample
# y_test_domain = np.concatenate((np.zeros(x_test1.shape[0]), np.ones(mnistm_test.shape[0])),axis=0)
#print(x_test_domain.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#one hot encode outputs
#y_train = np_utils.to_categorical(y_train, num_classes)
#y_test = np_utils.to_categorical(y_test, num_classes)

# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#bu hocanın ki
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='relu'))

# this is irrelevant for what we want to achieve
#model.compile(loss="mse", optimizer="adam")
# print("compilation is over")
nnw = NNWeightHelper(model)
weights = nnw.get_weights()

def main():
    print("Total number of weights to evolve is:", weights.shape)
    all_examples_indices = list(range(x_train.shape[0]))
    clf, _ = train_classifier(model, x_train, y_train)
    y_pred = predict_classifier(model, clf, x_test)
    print(y_test.shape, y_pred.shape)

    test_accuracy = accuracy_score(y_test, y_pred)
    print('Non-trained NN Test accuracy:', test_accuracy)
    # print('Test MSE:', test_mse)

    snes = SNES(weights, 1, POPULATION_SIZE)
    for i in range(0, GENERATIONS):
        start = timer()
        asked = snes.ask()

        # to be provided back to snes
        told = []

        # use a small number of training samples for speed purposes
        subsample_indices = np.random.choice(all_examples_indices, size=SAMPLE_SIZE, replace=False)
        # evaluate on another subset
        subsample_indices_valid = np.random.choice(all_examples_indices, size=SAMPLE_SIZE + 1, replace=False)

        # iterate over the population
        for asked_j in asked:
            # set nn weights
            nnw.set_weights(asked_j)
            # train the classifer and get back the predictions on the training data
            clf, _ = train_classifier(model, x_train[subsample_indices], y_train[subsample_indices])
            clf2, _ = train_classifier(model, x_train_domain[subsample_indices], y_train_domain[subsample_indices])

            # calculate the predictions on a different set
            y_pred = predict_classifier(model, clf, x_train[subsample_indices_valid])
            # score = accuracy_score(y_train[subsample_indices_valid], y_pred)
            loss_lab = mean_squared_error(y_train[subsample_indices_valid], y_pred)
            
            y_pred2 = predict_classifier(model, clf2, x_train_domain[subsample_indices_valid])
            score2 = accuracy_score(y_train_domain[subsample_indices_valid], y_pred2)
            loss_dom = mean_squared_error(y_train_domain[subsample_indices_valid], y_pred)
            # clf, _ = train_classifier(model, x_train, y_train)
            # y_pred = predict_classifier(model, clf, x_test)
            # score = accuracy_score(y_test, y_pred)
            # append to array of values that are to be returned
            #inverted = (-score2)
            total_loss = (loss_lab + (2*-loss_dom))# change this functions

            told.append(total)

        snes.tell(asked, told)
        end = timer()
        print("It took", end - start, "seconds to complete generation", i + 1)

    nnw.set_weights(snes.center)

    clf, _ = train_classifier(model, x_train, y_train)
    y_pred = predict_classifier(model, clf, x_test)


    print(y_test.shape, y_pred.shape)
    test_accuracy = accuracy_score(y_test, y_pred)


    print('Test accuracy:', test_accuracy)


if __name__ == '__main__':
    main()


x_train shape: (2817, 2352)
x_test shape: (498, 2352)
2817 train samples
498 test samples
Total number of weights to evolve is: (111562,)


ValueError: Error when checking : expected conv2d_1_input to have 4 dimensions, but got array with shape (2817, 2352)